In [2]:
import os
import numpy as np
import glob
import pandas as pd  
import random
import math as mp
import pickle
#import matplotlib.pyplot as plt
import tensorflow as tf
import time
from pickle import load
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from pyquaternion import Quaternion
%matplotlib inline
#import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow.keras.backend as K
from numpy import array
from numpy import hstack
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Conv1D, Input, Lambda
from tensorflow.compat.v1.keras.layers import CuDNNLSTM, CuDNNGRU
from tensorflow.keras.layers import Dense,Flatten,Dropout, MaxPooling1D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import GlobalMaxPooling1D, AveragePooling1D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


import toml
from math import floor

from ipynb.fs.full.V07_Utils  import *
from ipynb.fs.full.V07_Prepare_dataset import *

In [3]:
######################################################################
# Init Configuration
######################################################################

config_path = 'C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Code/V07_config.toml'
cfg = toml.load(config_path)

# General 
n_in_seq = cfg['n_in_seq_quat']
n_out_seq = cfg['n_out_seq_quat']
n_steps = cfg['n_steps'] 
LAT_in_rows = cfg['LAT_in_rows'] #6*16,66ms= 64ms

# model ID --> specify this for every new model
model_id = cfg['8_1_model_ID']
model_type = cfg['8_model_type']

# Directories 
Path_dataset01_test = cfg['Path_dataset01']
Path_results_dataset01_DESP_quat_model_id = cfg['Path_results_dataset01'] + model_type + "/" + model_id  + "/" 

# Tensorflow configuration
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))

Num GPUs Available:  0


In [4]:
def compute_pos_metrics(true_pos, preds_pos):
        # Compute Eucliden and angular distances
        euc_dists = np.linalg.norm(true_pos - preds_pos, axis=1)
        #print(euc_dists)
        # Mean Absolute Error (MAE)
        mae_euc = sum(euc_dists) / euc_dists.shape[0]
        # Root Mean Squared Error (RMSE)
        rmse_euc = np.sqrt((euc_dists ** 2).mean())
        return euc_dists, mae_euc, rmse_euc


def compute_quat_metrics(true_rot, preds_rot,true_rot_quat, preds_rot_quat):
        # Compute Eucliden and angular distances
        ang_dists = np.array([Quaternion.distance(q1, q2) for q1, q2 in zip(true_rot_quat, preds_rot_quat)]) #(w, x, y, z)
        euc_dists = np.linalg.norm(true_rot - preds_rot, axis=1)
        # Mean Absolute Error (MAE)
        #mae_ang = np.rad2deg(np.sum(ang_dists) / ang_dists.shape[0])
        #mae_euc = np.rad2deg(np.sum(euc_dists) / euc_dists.shape[0])
        
        mae_ang = (np.sum(ang_dists) / ang_dists.shape[0])
        mae_euc = (np.sum(euc_dists) / euc_dists.shape[0])
        
        # Root Mean Squared Error (RMSE)
        rmse_ang = np.rad2deg(np.sqrt((ang_dists ** 2).mean()))
        rmse_euc = np.rad2deg(np.sqrt((euc_dists ** 2).mean()))
        return euc_dists, ang_dists, mae_ang, mae_euc, rmse_ang, rmse_euc

def eval_quat(true_rot, preds_rot):
        true_rot_quat = np.array([Quaternion(q) for q in true_rot]) #(w, x, y, z)
        preds_rot_quat = np.array([Quaternion(q) for q in preds_rot]) #(w, x, y, z)    
        
        return compute_quat_metrics(true_rot, preds_rot,true_rot_quat, preds_rot_quat)

# Position

In [10]:
print("Look Ahead Time: " + str(LAT_in_rows))

for csv in get_csv_files(Path_dataset01_test, "full*cont"):
    basename = os.path.splitext(os.path.basename(csv))[0]
    print(csv)
    
    dataframe = load_df(csv)
    
    # define sequences
    seq1 = dataframe[['x']].to_numpy()
    seq2 = dataframe[['y']].to_numpy()
    seq3 = dataframe[['z']].to_numpy()
    
    seq1 = seq1.reshape((len(seq1), 1))
    seq2 = seq2.reshape((len(seq2), 1))
    seq3 = seq3.reshape((len(seq3), 1))
    
    # horizontally stack columns
    dataset = hstack((seq1, seq2, seq3)) 

    dataset_p1 = dataset[:-1]
    dataset_p2 = dataset[1:]
    delta = np.array([(p1-p2) for p1, p2 in zip(dataset_p1,dataset_p2)])
    
    
    sqt_prev = np.array([0,0,0])
    sqt2_prev = np.array([0,0,0])
    
    t = LAT_in_rows
    alphas = [0.496]
    
    for alpha in alphas: 
        prediction_result = []
        
        # Perform SE
        for pos in delta[:-LAT_in_rows]:

            f = alpha * t / (1 - alpha)

            sqt_x = (alpha * pos[0]) + ((1 - alpha) * sqt_prev[0])
            sqt_y = (alpha * pos[1]) + ((1 - alpha) * sqt_prev[1])
            sqt_z = (alpha * pos[2]) + ((1 - alpha) * sqt_prev[2])

            sqt2_x = (alpha * sqt_x) + ((1 - alpha) * sqt2_prev[0])
            sqt2_y = (alpha * sqt_y) + ((1 - alpha) * sqt2_prev[1])
            sqt2_z = (alpha * sqt_z) + ((1 - alpha) * sqt2_prev[2])

            pred_x = (2 + f) * sqt_x - (1 + f) * sqt2_x
            pred_y = (2 + f) * sqt_y - (1 + f) * sqt2_y
            pred_z = (2 + f) * sqt_z - (1 + f) * sqt2_z
            result = np.array([pred_x, pred_y, pred_z])

            sqt_prev = np.array([sqt_x,sqt_y,sqt_z])
            sqt2_prev = np.array([sqt2_x,sqt2_y,sqt2_z])

            prediction_result.append(result)

        dataset_pred = np.array(prediction_result)
        dataset_true = delta[LAT_in_rows:]
        
        # Create dataframe for prediction & gt per timestamp and concate results 
        df_preds_pos = pd.DataFrame(np.row_stack(dataset_pred), columns= cfg['pos_coords_pred'])
        df_true_pos = pd.DataFrame(np.row_stack(dataset_true), columns= cfg['pos_coords_true'])
        df_rot_result = pd.concat([df_preds_pos, df_true_pos], axis=1)
        df_rot_result.to_csv(os.path.join(Path_results_dataset01_DESP_quat_model_id + "/Traces", basename + '_result_pos_DESP.csv'), index=False)
        print("Single Predictions are saved")

        euc_dists, mae_euc, rmse_euc = compute_pos_metrics(dataset_pred, dataset_true)
        print('Number of dists: ' + str(euc_dists.shape[0]))
        print('MAE_euc position is : ' + str(mae_euc))
        print('RMSE_euc rotation is : ' + str(rmse_euc))
        print("----")
        
         ############ Save metrices
        result_one_experiment = list([[mae_euc,rmse_euc]])
        df_results = pd.DataFrame(result_one_experiment, columns=['mae_euc','rmse_euc'])
        df_results.to_csv(os.path.join(Path_results_dataset01_DESP_quat_model_id + "/Evaluation", basename + '_pos_evaluation.csv'), index=False)

        ############ Save angular distances
        dist_one_experiment = np.array(euc_dists).flatten()
        dists_one_experiment = pd.DataFrame(dist_one_experiment, columns=['euc_dists'])
        dists_one_experiment.to_csv(os.path.join(Path_results_dataset01_DESP_quat_model_id + "/Evaluation", basename + '_pos_dists.csv'), index=False)


Look Ahead Time: 5
C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Data/01_Dataset\1_full_continious.csv
Single Predictions are saved
Number of dists: 34993
MAE_euc position is : 0.0032673587898332562
RMSE_euc rotation is : 0.005060128711987785
----
C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Data/01_Dataset\2_full_continious.csv
Single Predictions are saved
Number of dists: 14993
MAE_euc position is : 0.003383198188416322
RMSE_euc rotation is : 0.00510993949822753
----
C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Data/01_Dataset\3_full_continious.csv
Single Predictions are saved
Number of dists: 44993
MAE_euc position is : 0.003138752567041892
RMSE_euc rotation is : 0.004705439366115982
----
C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Data/01_Dataset\4_full_continious.csv
Single Predictions are saved
Number of dists: 69993
MAE_euc position is : 0.00330702297781126
RMSE_euc rotation is : 0.005612973031276649
----
C:/Use

# Rotation

In [9]:
######################
# Quaternion Delta
######################

print("Look Ahead Time: " + str(LAT_in_rows))

for csv in get_csv_files(Path_dataset01_test, "5*full*cont"):
    print(csv)
    basename = os.path.splitext(os.path.basename(csv))[0]
    
    dataframe = load_df(csv)
    
    # define sequences
    seq1 = dataframe[['qw']].to_numpy()
    seq2 = dataframe[['qx']].to_numpy()
    seq3 = dataframe[['qy']].to_numpy()
    seq4 = dataframe[['qz']].to_numpy()
    
    seq1 = seq1.reshape((len(seq1), 1))
    seq2 = seq2.reshape((len(seq2), 1))
    seq3 = seq3.reshape((len(seq3), 1))
    seq4 = seq4.reshape((len(seq4), 1))
    
    # horizontally stack columns
    dataset = hstack((seq1, seq2, seq3, seq4)) 
    
    
    dataset_q1 = ([Quaternion(q) for q in dataset[:-1]])
    dataset_q2 = ([Quaternion(q) for q in dataset[1:]])
    delta = np.array([(q1/q2).elements for q1, q2 in zip(dataset_q1,dataset_q2)])
    
    t = LAT_in_rows
    alphas = [0.61]
    
    for alpha in alphas: 
        
        print(alpha)
        
        prediction_result = []
        continious_dataset = []
        state_changes = []
        
        sqt_prev = np.array([0,0,0,0])
        sqt2_prev = np.array([0,0,0,0])
        
        ##################################################################################################
        # DESP
        ##################################################################################################
        
        for rot in delta[:-LAT_in_rows]:
        #for rot in dataset[:10]:
            
            
            #print("rot: " + str(rot))
            #print("-Ticke----------")

            f = alpha * t / (1 - alpha)
            

            sqt_w = (alpha * rot[0]) + ((f - alpha) * sqt_prev[0])
            sqt_x = (alpha * rot[1]) + ((f - alpha) * sqt_prev[1])
            sqt_y = (alpha * rot[2]) + ((f - alpha) * sqt_prev[2])
            sqt_z = (alpha * rot[3]) + ((f - alpha) * sqt_prev[3])

            sqt2_w = (alpha * sqt_w) + ((f - alpha) * sqt2_prev[0])
            sqt2_x = (alpha * sqt_x) + ((f - alpha) * sqt2_prev[1])
            sqt2_y = (alpha * sqt_y) + ((f - alpha) * sqt2_prev[2])
            sqt2_z = (alpha * sqt_z) + ((f - alpha) * sqt2_prev[3])

            q_pred_w = (2 + f) * sqt_w - (1 + f) * sqt2_w;
            q_pred_x = (2 + f) * sqt_x - (1 + f) * sqt2_x;
            q_pred_y = (2 + f) * sqt_y - (1 + f) * sqt2_y;
            q_pred_z = (2 + f) * sqt_z - (1 + f) * sqt2_z;
            
            #Quaternion_result_norm = np.linalg.norm(np.array([q_pred_w, q_pred_x, q_pred_y, q_pred_z]))
            #Quaternion_result = np.array([q_pred_w, q_pred_x, q_pred_y, q_pred_z])/Quaternion_result_norm
            
            q_res_unit = Quaternion([q_pred_w, q_pred_x, q_pred_y, q_pred_z]).unit
            q_res_unit = np.array([q_res_unit[0],q_res_unit[1],q_res_unit[2],q_res_unit[3]])
            
            #print(q_res_unit)
            
            sqt_prev = Quaternion([sqt_w,sqt_x,sqt_y,sqt_z]).unit
            sqt_prev = np.array([sqt_prev[0],sqt_prev[1],sqt_prev[2],sqt_prev[3]])
            #print(sqt_prev)            
            
            sqt2_prev_normal = np.array([sqt2_w,sqt2_x,sqt2_y,sqt2_z])
            #print("normal: " + str(sqt2_prev_normal))
            sqt2_prev = Quaternion([sqt2_w,sqt2_x,sqt2_y,sqt2_z]).unit
            #print("Quat: " + str(sqt2_prev))
            sqt2_prev = np.array([sqt2_prev[0],sqt2_prev[1],sqt2_prev[2],sqt2_prev[3]])
            #print("After :" + str(sqt2_prev))
            #print("-----")
            
            
            
            #sqt_prev_norm = np.linalg.norm(np.array([sqt_w,sqt_x,sqt_y,sqt_z]))
            #sqt_prev = np.array([sqt_w,sqt_x,sqt_y,sqt_z])/sqt_prev_norm
            
            #sqt2_prev_norm = np.linalg.norm(np.array([sqt2_w,sqt2_x,sqt2_y,sqt2_z]))
            #sqt2_prev = np.array([sqt2_w,sqt2_x,sqt2_y,sqt2_z])/sqt2_prev_norm
            
            prediction_result.append(q_res_unit)
        
        ##################################################################################################
        # Checking continiousity of predicted data
        ##################################################################################################
        
        dataset_pred = np.array(prediction_result)
        dataset_pred = np.array([Quaternion(q) for q in dataset_pred]) 
        dataset_true = delta[LAT_in_rows:]
        
        
        
        def representation_1(q):
            quaternion = []
            quaternion.append([q[0],q[1],q[2],q[3]])
            return np.array(quaternion).reshape(4,)
    
        def representation_2(q):
            quaternion = []
            quaternion.append([-q[0],-q[1],-q[2],-q[3]])
            return np.array(quaternion).reshape(4,)

        for q1,q2 in zip(dataset_pred[:-1],dataset_pred[1:]):
            if Quaternion.distance(q1,q2) <= Quaternion.distance(q1,-q2):
                state_changes.append(0)
            else:
                state_changes.append(1)

        cum_sum = np.cumsum(np.array(state_changes))

        for idx,state in enumerate(cum_sum):
            if state%2 == 0:
                continious_dataset.append(representation_2(dataset_pred[idx+1]))
            else:
                continious_dataset.append(representation_1(dataset_pred[idx+1]))

        
        #print("Bevor")
        #print(dataset_pred)
        #print("Groundtruth")
        #print(dataset_true)
        #print("after")
        #print(np.array(continious_dataset))
        
        
        euc_dists, ang_dists, mae_ang, mae_euc, rmse_ang, rmse_euc = eval_quat(continious_dataset, dataset_true[:-1])
        
        
        # Create dataframe for prediction & gt per timestamp and concate results 
        df_preds_rot = pd.DataFrame(np.row_stack(continious_dataset), columns= cfg['quat_coords_pred'])
        df_true_rot = pd.DataFrame(np.row_stack(dataset_true[:-1]), columns= cfg['quat_coords_true'])
        df_rot_result = pd.concat([df_preds_rot, df_true_rot], axis=1)
        df_rot_result.to_csv(os.path.join(Path_results_dataset01_DESP_quat_model_id + "/Traces", basename + '_delta_result_quat_DESP.csv'), index=False)
        print("Single Predictions are saved")
        
        print('Number of dists: ' + str(euc_dists.shape[0]))
        print('MAE_euc position is : ' + str(mae_euc))
        print('RMSE_euc rotation is : ' + str(rmse_euc))
        
         ############ Save metrices
        result_one_experiment = list([[mae_ang,mae_euc,rmse_ang,rmse_euc]])
        df_results = pd.DataFrame(result_one_experiment, columns=['mae_ang','mae_euc','rmse_ang','rmse_euc'])
        df_results.to_csv(os.path.join(Path_results_dataset01_DESP_quat_model_id + "/Evaluation", basename + '_delta_quat_evaluation.csv'), index=False)

        ############ Save angular distances
        dist_one_experiment = list(ang_dists)
        dists_one_experiment = pd.DataFrame(dist_one_experiment, columns=['ang_dists'])
        dists_one_experiment.to_csv(os.path.join(Path_results_dataset01_DESP_quat_model_id + "/Evaluation", basename + '_delta_quat_dists.csv'), index=False)
        
        
        
        print("----")
        print("----")


Look Ahead Time: 5
C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Data/01_Dataset\5_full_continious.csv
0.61
Single Predictions are saved
Number of dists: 19992
MAE_euc position is : 0.00496613254378738
RMSE_euc rotation is : 0.4284428668842418
----
----


In [ ]:
######################
# Quaternion Absolut
######################



print("Look Ahead Time: " + str(LAT_in_rows))

for csv in get_csv_files(Path_dataset01_test, "full*cont"):
    print(csv)
    basename = os.path.splitext(os.path.basename(csv))[0]
    
    dataframe = load_df(csv)
    
    # define sequences
    seq1 = dataframe[['qw']].to_numpy()
    seq2 = dataframe[['qx']].to_numpy()
    seq3 = dataframe[['qy']].to_numpy()
    seq4 = dataframe[['qz']].to_numpy()
    
    seq1 = seq1.reshape((len(seq1), 1))
    seq2 = seq2.reshape((len(seq2), 1))
    seq3 = seq3.reshape((len(seq3), 1))
    seq4 = seq4.reshape((len(seq4), 1))
    
    # horizontally stack columns
    dataset = hstack((seq1, seq2, seq3, seq4)) 
    
    
    #dataset_q1 = ([Quaternion(q) for q in dataset[:-1]])
    #dataset_q2 = ([Quaternion(q) for q in dataset[1:]])
    #delta = np.array([(q1/q2).elements for q1, q2 in zip(dataset_q1,dataset_q2)])
    
    t = LAT_in_rows
    alphas = [0.61]
    
    for alpha in alphas: 
        
        print(alpha)
        
        prediction_result = []
        continious_dataset = []
        state_changes = []
        
        sqt_prev = np.array([0,0,0,0])
        sqt2_prev = np.array([0,0,0,0])
        
        ##################################################################################################
        # DESP
        ##################################################################################################
        
        for rot in dataset[:-LAT_in_rows]:
        #for rot in dataset[:10]:
            
            
            #print("rot: " + str(rot))
            #print("-Ticke----------")

            f = alpha * t / (1 - alpha)
            

            sqt_w = (alpha * rot[0]) + ((f - alpha) * sqt_prev[0])
            sqt_x = (alpha * rot[1]) + ((f - alpha) * sqt_prev[1])
            sqt_y = (alpha * rot[2]) + ((f - alpha) * sqt_prev[2])
            sqt_z = (alpha * rot[3]) + ((f - alpha) * sqt_prev[3])

            sqt2_w = (alpha * sqt_w) + ((f - alpha) * sqt2_prev[0])
            sqt2_x = (alpha * sqt_x) + ((f - alpha) * sqt2_prev[1])
            sqt2_y = (alpha * sqt_y) + ((f - alpha) * sqt2_prev[2])
            sqt2_z = (alpha * sqt_z) + ((f - alpha) * sqt2_prev[3])

            q_pred_w = (2 + f) * sqt_w - (1 + f) * sqt2_w;
            q_pred_x = (2 + f) * sqt_x - (1 + f) * sqt2_x;
            q_pred_y = (2 + f) * sqt_y - (1 + f) * sqt2_y;
            q_pred_z = (2 + f) * sqt_z - (1 + f) * sqt2_z;
            
            #Quaternion_result_norm = np.linalg.norm(np.array([q_pred_w, q_pred_x, q_pred_y, q_pred_z]))
            #Quaternion_result = np.array([q_pred_w, q_pred_x, q_pred_y, q_pred_z])/Quaternion_result_norm
            
            q_res_unit = Quaternion([q_pred_w, q_pred_x, q_pred_y, q_pred_z]).unit
            q_res_unit = np.array([q_res_unit[0],q_res_unit[1],q_res_unit[2],q_res_unit[3]])
            
            #print(q_res_unit)
            
            sqt_prev = Quaternion([sqt_w,sqt_x,sqt_y,sqt_z]).unit
            sqt_prev = np.array([sqt_prev[0],sqt_prev[1],sqt_prev[2],sqt_prev[3]])
            #print(sqt_prev)            
            
            sqt2_prev_normal = np.array([sqt2_w,sqt2_x,sqt2_y,sqt2_z])
            #print("normal: " + str(sqt2_prev_normal))
            sqt2_prev = Quaternion([sqt2_w,sqt2_x,sqt2_y,sqt2_z]).unit
            #print("Quat: " + str(sqt2_prev))
            sqt2_prev = np.array([sqt2_prev[0],sqt2_prev[1],sqt2_prev[2],sqt2_prev[3]])
            #print("After :" + str(sqt2_prev))
            #print("-----")
            
            
            
            #sqt_prev_norm = np.linalg.norm(np.array([sqt_w,sqt_x,sqt_y,sqt_z]))
            #sqt_prev = np.array([sqt_w,sqt_x,sqt_y,sqt_z])/sqt_prev_norm
            
            #sqt2_prev_norm = np.linalg.norm(np.array([sqt2_w,sqt2_x,sqt2_y,sqt2_z]))
            #sqt2_prev = np.array([sqt2_w,sqt2_x,sqt2_y,sqt2_z])/sqt2_prev_norm
            
            prediction_result.append(q_res_unit)
        
        ##################################################################################################
        # Checking continiousity of predicted data
        ##################################################################################################
        
        dataset_pred = np.array(prediction_result)
        dataset_pred = np.array([Quaternion(q) for q in dataset_pred]) 
        dataset_true = dataset[LAT_in_rows:]
        
        
        
        def representation_1(q):
            quaternion = []
            quaternion.append([q[0],q[1],q[2],q[3]])
            return np.array(quaternion).reshape(4,)
    
        def representation_2(q):
            quaternion = []
            quaternion.append([-q[0],-q[1],-q[2],-q[3]])
            return np.array(quaternion).reshape(4,)

        for q1,q2 in zip(dataset_pred[:-1],dataset_pred[1:]):
            if Quaternion.distance(q1,q2) <= Quaternion.distance(q1,-q2):
                state_changes.append(0)
            else:
                state_changes.append(1)

        cum_sum = np.cumsum(np.array(state_changes))

        for idx,state in enumerate(cum_sum):
            if state%2 == 0:
                continious_dataset.append(representation_1(dataset_pred[idx+1]))
            else:
                continious_dataset.append(representation_2(dataset_pred[idx+1]))

        
        #print("Bevor")
        #print(dataset_pred)
        #print("Groundtruth")
        #print(dataset_true)
        #print("after")
        #print(np.array(continious_dataset))
        
        
        euc_dists, ang_dists, mae_ang, mae_euc, rmse_ang, rmse_euc = eval_quat(continious_dataset, dataset_true[:-1])
        
        
        # Create dataframe for prediction & gt per timestamp and concate results 
        df_preds_rot = pd.DataFrame(np.row_stack(continious_dataset), columns= cfg['quat_coords_pred'])
        df_true_rot = pd.DataFrame(np.row_stack(dataset_true[:-1]), columns= cfg['quat_coords_true'])
        df_rot_result = pd.concat([df_preds_rot, df_true_rot], axis=1)
        df_rot_result.to_csv(os.path.join(Path_results_dataset01_DESP_quat_model_id + "/Traces", basename + '_result_quat_DESP.csv'), index=False)
        print("Single Predictions are saved")
        
        print('Number of dists: ' + str(euc_dists.shape[0]))
        print('MAE_euc position is : ' + str(mae_euc))
        print('RMSE_euc rotation is : ' + str(rmse_euc))
        
         ############ Save metrices
        result_one_experiment = list([[mae_ang,mae_euc,rmse_ang,rmse_euc]])
        df_results = pd.DataFrame(result_one_experiment, columns=['mae_ang','mae_euc','rmse_ang','rmse_euc'])
        df_results.to_csv(os.path.join(Path_results_dataset01_DESP_quat_model_id + "/Evaluation", basename + '_quat_evaluation.csv'), index=False)

        ############ Save angular distances
        dist_one_experiment = list([[ang_dists,euc_dists]])
        dists_one_experiment = pd.DataFrame(dist_one_experiment, columns=['ang_dists','euc_dists'])
        dists_one_experiment.to_csv(os.path.join(Path_results_dataset01_DESP_quat_model_id + "/Evaluation", basename + '_quat_dists.csv'), index=False)
        
        
        
        print("----")
        print("----")


In [466]:
print(dataset[1])

Quaternion_result_norm = np.linalg.norm(np.array([0.02956371 , 0.00830975 , 0.996813   , 0.07362611]))
Quaternion_result_norm2 = np.linalg.norm([0.02956371 , 0.00830975 , 0.996813   , 0.07362611])

Quaternion_result = np.array([q_pred_w, q_pred_x, q_pred_y, q_pred_z])/Quaternion_result_norm
Quaternion_result2 = np.array([q_pred_w, q_pred_x, q_pred_y, q_pred_z])/Quaternion_result_norm

print(Quaternion_result)
print(Quaternion_result2)

unit = Quaternion().unit
print(unit)

arr = np.array([unit[0],unit[1],unit[2],unit[3]])
print(arr)

SyntaxError: invalid syntax (<ipython-input-466-f448a2bfa24b>, line 12)

In [ ]:
    
        
        
        
        
        
        dataset_pred = np.array([Quaternion(q) for q in dataset_pred]) 
        dataset_true = np.array([Quaternion(q) for q in dataset_true])
        
        
        
        def representation_1(q):
            quaternion = []
            quaternion.append([q[0],q[1],q[2],q[3]])
            return np.array(quaternion).reshape(4,)

        def representation_2(q):
            quaternion = []
            quaternion.append([-q[0],-q[1],-q[2],-q[3]])
            return np.array(quaternion).reshape(4,)
        
        
        continious_dataset_pred = []
        state_changes = []
        
        for yhat,ytrue in zip(dataset_pred,dataset_true):
            
            print("q1 = " + str( Quaternion.distance(yhat,ytrue)))
            print("q2 = " + str( Quaternion.distance(-yhat,ytrue)))
            
            if Quaternion.distance(yhat,ytrue) <= Quaternion.distance(-yhat,ytrue):
                state_changes.append(0)
            else:
                state_changes.append(1)

        cum_sum = np.cumsum(np.array(state_changes))
        print(cum_sum)

        for idx,state in enumerate(cum_sum):

            if state%2 == 0:
                continious_dataset_pred.append(representation_1(dataset_pred[idx]))
            else:
                continious_dataset_pred.append(representation_2(dataset_pred[idx]))
        
        continious_dataset_pred = np.array(continious_dataset_pred)
        print(continious_dataset_pred)
        dataset_true = dataset[LAT_in_rows:]

In [ ]:
quat = np.array([Quaternion(q) for q in dataset]) 
continious_dataset = []
state_changes = []
dist = []

def representation_1(q):
    quaternion = []
    quaternion.append([q[0],q[1],q[2],q[3]])
    return np.array(quaternion).reshape(4,)
    
def representation_2(q):
    quaternion = []
    quaternion.append([-q[0],-q[1],-q[2],-q[3]])
    return np.array(quaternion).reshape(4,)

for yhat,ytrue in zip(dataset_pred,dataset_true):
    if Quaternion.distance(yhat,ytrue) <= Quaternion.distance(yhat,ytrue):
        state_changes.append(0)
        dist.append(Quaternion.distance(q1,q2))
    else:
        state_changes.append(1)
        dist.append(Quaternion.distance(q1,-q2))

cum_sum = np.cumsum(np.array(state_changes))

    
for idx,state in enumerate(cum_sum):
    
    if state%2 == 0:
        continious_dataset.append(representation_1(quat[idx+1]))
    else:
        continious_dataset.append(representation_2(quat[idx+1]))

In [12]:
alpha = 0.45

for pos in dataset:
    pos_sqt_prev = pos
    pos_sqt2_prev = pos
    
    result = PredictRotation()
    #rot_sqt_prev 
    #rot_sqt2_prev  

In [ ]:
public Vector3 PredictPosition(Vector3 pos, int t)
{
    float f = alpha * t / (1f - alpha);

    Vector3 sqt = Vector3.zero;
    sqt.x = (alpha * pos.x) + ((1f - alpha) * vec3_sqt_prev.x);
    sqt.y = (alpha * pos.y) + ((1f - alpha) * vec3_sqt_prev.y);
    sqt.z = (alpha * pos.z) + ((1f - alpha) * vec3_sqt_prev.z);

    Vector3 sqt2 = Vector3.zero;
    sqt2.x = (alpha * sqt.x) + ((1f - alpha) * vec3_sqt2_prev.x);
    sqt2.y = (alpha * sqt.y) + ((1f - alpha) * vec3_sqt2_prev.y);
    sqt2.z = (alpha * sqt.z) + ((1f - alpha) * vec3_sqt2_prev.z);

    Vector3 vec3_pred = Vector3.zero;
    vec3_pred.x = (2f + f) * sqt.x - (1f + f) * sqt2.x;
    vec3_pred.y = (2f + f) * sqt.y - (1f + f) * sqt2.y;
    vec3_pred.z = (2f + f) * sqt.z - (1f + f) * sqt2.z;
    Vector3 result = new Vector3(vec3_pred.x, vec3_pred.y, vec3_pred.z);

    vec3_sqt_prev = sqt;
    vec3_sqt2_prev = sqt2;
    return result;
}
